In [1]:
import numpy as np
import requests
import json
from datetime import datetime
from collections import deque
import inspect

# Block-driven architecture 

In [2]:
def get_blocks_heights_in_last_n_hours(n: int):
    
    response = requests.get(url='https://api.bitaps.com/btc/v1/blockchain/blocks/last/'+str(n)+'/hours')
    json_data = json.loads(response.text)
    data = json_data['data']
    
    blocks_heights = [element['height'] for element in data]
    
    return blocks_heights

In [37]:
def initialize_metrics(size_in_blocks):
    
    fee_queue = deque([0]*size_in_blocks, size_in_blocks)
    block_pow_queue = deque([600]*size_in_blocks, size_in_blocks)
    cval_queue = deque([0]*size_in_blocks, size_in_blocks)
    txs_queue = deque([0]*size_in_blocks, size_in_blocks)
    unsigned_txs_queue = deque([0]*size_in_blocks, size_in_blocks)
    
    queues = {
        'fee':fee_queue,
        'block_pow':block_pow_queue,
        'cval':cval_queue,
        'txs_count':txs_queue,
        'unsigned_txs':unsigned_txs_queue
    } 
    
    metrics = {
        'fee':{'avg':0, 'min':np.inf, 'max':-np.inf, 'sum':0},
        'block_pow':{'tau':600, 'avg':0, 'min':np.inf, 'max':-np.inf, 'sum':600*size_in_blocks},
        'cval':{'avg':0, 'min':np.inf, 'max':-np.inf, 'sum':0},
        'txs_count':{'avg':0, 'min':np.inf, 'max':-np.inf, 'sum':0},
        'unsigned_txs':{'avg':0, 'min':np.inf, 'max':-np.inf, 'sum':0}
    }
    
    metrics_dict = {
        'queues': queues,
        'metrics': metrics,
        'size':size_in_blocks,
    }
    
    return metrics_dict

In [29]:
heights = get_blocks_heights_in_last_n_hours(2)

In [38]:
metrics = initialize_metrics(5)

In [42]:
def update_metrics(old_metrics: dict, new_blocks_heights: list):
    
        
    def get_name(variable):

#         for fi in reversed(inspect.stack()):
#             names = [var_name for var_name, var_val in fi.frame.f_locals.items() if var_val is var]
#             if len(names) > 0:
#                 return names[0]
#         return (list(globals().keys()))[list(map(lambda x: id(x), list(globals().values()))).index(id(variable))]
        return(Retriever(variable))
            
    def get_unsigned_transaction_number():
    
        response = requests.get(url="https://blockchain.info/q/unconfirmedcount")
        count = json.loads(response.text)

        return count
    
    def get_block_info(block_height: int):
    
        url = 'https://api.blockchair.com/bitcoin/blocks?q=id(' + str(block_height) + ')'

        response = requests.get(url=url)
        json_data = json.loads(response.text)
        block = json_data['data'][0]

        # for Average interval between blocks pow
        time = datetime.strptime(block['time'],'%Y-%m-%d %H:%M:%S')
        # Miners fees
        fee = (block['fee_total'], block['fee_total_usd'])
        # Number of Transactions in a block
        txs_count = block['transaction_count']
        # Crypto-value in a block
        input_total = (block['input_total'],block['input_total_usd'])

        result = {
            'time':time,
#             'fee':{
#                 'satoshi':fee[0],
#                 'usd':fee[1]
#             },
            'fee':fee[0],
            'txs_count':txs_count,
#             'input_total':{
#                 'satoshi':input_total[0],
#                 'usd':input_total[1]
#             }
            'input_total':input_total[0]
        }

        return result


    def update_metrics_dict(new_metrics: list, metrics_dict:dict):

        name_map = {
            'fee':{'queues':'fee', 'metrics':'fee'},
            'time':{'queues':'block_pow', 'metrics':'block_pow'},
            'inpit_total':{'queues':'cval', 'metrics':'cval'},
            'txs_count':{'queues':'txs_count', 'metrics':'txs_count'},
            'unsigned_txs':{'queues':'unsigned_txs', 'metrics':'unsigned_txs'}
        }
        
        metrics_names_list = ['fee', 'time', 'inpit_total', 'txs_count', 'unsigned_txs']
        

        for idx, metric in enumerate(new_metrics):

            metric_name = metrics_names_list[idx]
            queue_name_map = name_map[metric_name]['queues']
            metric_name_map = name_map[metric_name]['metrics']
            metrics = metrics_dict['metrics'][metric_name_map]
            size = metrics_dict['size']

            poped = metrics_dict['queues'][queue_name_map].popleft()
            metrics_dict['queues'][queue_name_map].append(metric)
            metrics['sum'] -= poped
            metrics['sum'] += metric
            if metric < metrics['min']: 
                metrics['min'] = metric
            if metric > metrics['max']:
                metrics['max'] = metric
            metrics['avg'] = metrics['sum'] / size

            if 'tau' in metrics.keys():

                tau = metrics['tau']
                
                denominator = size / tau
                denominator += (1/metric - 1/poped)
                tau = size / denominator

                metrics['tau'] = tau

            metrics_dict['metrics'][metric_name_map] = metrics

        return metrics_dict


    for block_height in sorted(new_blocks_heights, reverse=False):
        
        print(block_height)
        
        block_info = get_block_info(block_height)
        
        fee = block_info['fee']
        time = (block_info['time'] - get_block_info(block_height-1)['time']).seconds
        inpit_total = block_info['input_total']
        txs_count = block_info['txs_count']
        unsigned_txs = get_unsigned_transaction_number()
        
        new_metrics = [fee, time, inpit_total, txs_count, unsigned_txs]
        
        updated_metrics_dict = update_metrics_dict(new_metrics, old_metrics)
        
    return updated_metrics_dict
        
    

In [40]:
updated = update_metrics(metrics, heights)

615654
0.008333333333333333 0.0005252100840336134 0.0016666666666666668
0.0071918767507002795 695.2288218111004
615655
0.0071918767507002795 0.0008496176720475786 0.0016666666666666668
0.006374827756081191 784.3349171638888
615656
0.006374827756081191 0.0011086474501108647 0.0016666666666666668
0.005816808539525389 859.5778881193784
615657
0.005816808539525389 0.0008453085376162299 0.0016666666666666668
0.004995450410474953 1000.9107466096565
615658
0.004995450410474953 0.0037593984962406013 0.0016666666666666668
0.007088182240048887 705.3994706498285
615659
0.007088182240048886 0.0010718113612004287 0.0005252100840336134
0.007634783517215702 654.8974163740834
615660
0.007634783517215702 0.005747126436781609 0.0008496176720475786
0.012532292281949731 398.9693096450921
615661
0.012532292281949731 0.01282051282051282 0.0011086474501108647
0.02424415765235169 206.235253527771
615662
0.02424415765235169 0.00129366106080207 0.0008453085376162299
0.02469251017553753 202.49055136376614
615663

In [41]:
updated

{'queues': {'fee': deque([19248652, 15030892, 10560682, 18028012, 15206193]),
  'block_pow': deque([933, 174, 78, 773, 452]),
  'cval': deque([650724388913,
         376452494839,
         239892495224,
         400329241164,
         249217147309]),
  'txs_count': deque([2874, 2364, 2070, 2499, 1934]),
  'unsigned_txs': deque([10071, 10071, 10071, 10071, 10071])},
 'metrics': {'fee': {'avg': 15614886.2,
   'min': 10560682,
   'max': 75705553,
   'sum': 78074431},
  'block_pow': {'tau': 216.02470333546444,
   'avg': 482.0,
   'min': 78,
   'max': 1904,
   'sum': 2410},
  'cval': {'avg': 383323153489.8,
   'min': 239892495224,
   'max': 1210569201846,
   'sum': 1916615767449},
  'txs_count': {'avg': 2348.2, 'min': 980, 'max': 2874, 'sum': 11741},
  'unsigned_txs': {'avg': 10071.0, 'min': 5975, 'max': 10071, 'sum': 50355}},
 'size': 5}

# Unused 

In [23]:
def get_block_info(block_height: int):
    
    url = 'https://api.blockchair.com/bitcoin/blocks?q=id(' + str(block_height) + ')'

    response = requests.get(url=url)
    json_data = json.loads(response.text)
    block = json_data['data'][0]

    # for Average interval between blocks pow
    time = datetime.strptime(block['time'],'%Y-%m-%d %H:%M:%S')
    # Miners fees
    fee = (block['fee_total'], block['fee_total_usd'])
    # Number of Transactions in a block
    txs_count = block['transaction_count']
    # Crypto-value in a block
    input_total = (block['input_total'],block['input_total_usd'])

    result = {
        'time':time,
        'fee':{'satoshi':fee[0],'usd':fee[1]},
        'txs_count':txs_count,
        'input_total':{'satoshi':input_total[0],
        'usd':input_total[1]}
    }

    return result



In [34]:
block_info = get_block_info(615650)
time2 = block_info['time']

In [36]:
(block_info['time'] - get_block_info(615649)['time']).seconds

36

In [33]:
z.seconds

277

In [5]:
def update_metrics(new_data:dict, times: tuple, data: dict, metrics:dict, unsigned_txs_val):
    # all inputs exept new_data are dqueues with data in them
    #previuos_data is miners_fees_queue,block_pow_queue,block_trans_amaount_queue,cr_value_queue,unsigned_transactions
    #previous_metrics are metrics
    # store metrics data, so no need to calculate every time (previuos_metrics element)
    first_block_time , last_block_time = times
    for key in sorted(new_data.keys()):
        time = new_data[key]['time']
        if first_block_time == time: continue
        data = new_data[key]
        
    def update(new_data:dict, data:dict, previous_metrics:dict, last_block_time, unsigned_txs_val):
#         poped_fee = metrics['fees_queue'].popleft()
        time = new_data['time']
        block_time = time - last_block_time
        
        pop_insert_map_dict = {
                    'fee':'fees_queue',
                    'txs_count':'txs_count_queue',
                    'input_total':'cval_queue',
                    'block_time':'block_pow_queue',
                    'unsigned_txs_val':'unsigned_txs_queue'
                   }
        
        def pop_insert(new_data, data, block_time, unsigned_txs_val, pop_insert_map_dict):
            poped = dict
            new_data['block_time'] = block_time
            new_data['unsigned_txs_val'] = unsigned_txs_val
            for key, value in map_dict.items():
                poped[value] = data[value].popleft()
                data[value].append(new_data[key])
                
            return poped, data
        
        calculate_map_dict={
                'fees_queue':'miners_fees',
                'txs_queue':'block_txs_count',
                'cval_queue':'crval',
                'block_pow_queue':'block_pow',
                'unsigned_txs_queue':'unsigned_txs'
                }
        
        def calculate(poped, previous_metrics, map_dict, queues):
            for key, value in poped.items():
                queue = queues[key]
                process_metric(value, previous_metrics, queue)
            # {'tau':0, 'avg':0,'min':np.inf,'max':-np.inf, 'sum':0},
            def process_metric(poped_value, metric:dict, queue:deque):
                metric['sum'] -= poped_value
                unseen_element = queue[-1]
                metric['sum'] += unseen_element
                if unseen_element > metric['max']: metric['max'] = unseen_element
                if unseen_element < metric['min']: metric['min'] = unseen_element
                avg = np.mean(queue)
                metric['tau'] = calculate_tau(queue)
                
                return metric
            
                def calculate_tau(queue):
                    numerator = len(queue)
                    denominator = 0
                    for element in queue:
                        denominator += 1 / element
                return numerator / denominator
        
        metrics['fees_queue'].append(new_data['fee'])        
        
        

        metrics['block_pow_queue'].append(block_time)
        metrics['block_txs_amount_queue'].append(new_data['txs_count'])
        metrics['cval_queue'].append(new_data['input_total'])
        metrics['txs_queue'].append(new_data['txs_count'])
        # do i need value of unsigned trans???? 
        
        ## TODO update metrics
        metrics['last_block_index'] = key
        fee = metrics['miners_fees'].popleft()
        
        

In [70]:
a = deque([1,2,3],3)

In [72]:
a.append(8)

In [73]:
np.mean(a)

4.333333333333333

In [36]:
a = {'e':[3,4]}

In [37]:
a['e'].append(5)

In [38]:
a['e']

[3, 4, 5]

In [7]:
def calculate_time_delta(date1, date2):
    return (date1 - date2).total_seconds()

In [8]:
def calculate_tau(queue):
    numerator = len(queue)
    denominator = 0
    for element in queue:
        denominator += 1 / element
    return numerator / denominator

In [64]:
sorted([4,1,2],reverse=True)

[4, 2, 1]

In [26]:
new_data, first_time, last_time = get_blockchain_info(2)

In [28]:
first_time

datetime.datetime(2019, 11, 6, 15, 25, 38)

In [17]:
get_unsigned_transactions()

{'196f42aba2f819e2f28a3688980e55cae1c4a39bb4cfc8e353b246138edb6239': (1591864,
  8136),
 '97d220a2ff9af685889074b05a82f4152e6433642b4f93fe78b1f096f446f979': (1958141,
  4914),
 'edaf3bde1ba11a988d6d3e6792efba818389633cb7f6a2ae426127f6b97403d4': (8864026,
  11300),
 'f19e74d5c8672b920530a4e5940a425f8e4cf6a8fcee07b2a0cc2625ab2d8b53': (11089617,
  5401),
 '554096f90c407a4a0f34357af2f93a0bff80f5cec1ae34d312d37f8730fbc471': (33494557,
  2486),
 '440967e1f8b88b08dab7f8bd903d000502295160dfcbfd07ac1b8bcfd1764bf1': (512764,
  18180),
 '0d38ee5f95f49aeb7af1e6fbfc0217bd1616c2fecf5ca64ca4f07256471dd0cf': (195095547,
  11707),
 '41f6e3673539ce705e2cef6f9bcc18cb386975fe9d40cc4d30ddc373cd28499a': (57267956,
  11685),
 '19e59fd0c1376b2dda33479c78aeaa29992dc41e38c546ca4689f16ea254b687': (231169,
  2497),
 '1987e810bd65b6f26576faff4592fe989aba4c3fdd41b606dc2ff96d030e7052': (79579,
  4903),
 'd0157c02c79373b267605581aeb67f78aa540dab930d517f4751f2e046cccd41': (118729952,
  4278),
 '7098d1051d11803e4d541f4

In [ ]:
def calculate

In [ ]:
def checker # depens on first_block_time or last

In [30]:
# initiate all metrics as some values, not compute every time, just check, like:
# if val>maxval => maxval = val ...

In [37]:
def get_unsigned_transactions():
    response = requests.get(url='https://api.bitaps.com/btc/v1/mempool/transactions')
    json_data = json.loads(response.text)
    transactions = json_data['data']['transactions']
#     transactions_dict = dict()
    total = 0
    for transaction in transactions:
        total += amount
#         txId = transaction['txId']
#         amount = transaction['amount']
#         fee = transaction['fee']
#         transactions_dict[txId] = (amount, fee)
    transactions_dict['total'] = total
    
    return total

In [4]:
# Rewrote as get_blocks_heights_in_last_n_hours(n)

def get_blockchain_info(hours):
    response = requests.get(url='https://api.bitaps.com/btc/v1/blockchain/blocks/last/'+str(hours)+'/hours')
    json_data = json.loads(response.text)
    data = json_data['data']
    new_data = dict()
    for element in data:
        height = element['height']
        new_data[height] = get_block_info(height)
    first_block_time = new_data[sorted(new_data.keys(),reverse=False)[0]]['time']    
    last_block_time = new_data[sorted(new_data.keys(),reverse=True)[0]]['time']
    
    
    return new_data , first_block_time, last_block_time

In [6]:
# Rewrote with the same name

def initiate_metrics(size_in_hours):
    # size is a tuple with miners_fees_queue_size, block_pow_queue_size, etc.
    def calculate_queue_sizes(hours):
        AVG_POW_IN_HOUR = 6
        size = hours * AVG_POW_IN_HOUR 
        return size
    q_sizes = calculate_queue_sizes(size_in_hours)
    
    fees_queue = deque([0], q_sizes)
    block_pow_queue = deque([0], q_sizes)
    crval_queue = deque([0], q_sizes)
    txs_queue = deque([0], q_sizes)
    unsigned_txs_queue = deque([0], q_sizes)
    queues = {'fees_queue':miners_fees_queue,
              'block_pow_queue':block_pow_queue,
              'cval_queue':cval_queue,
              'txs_count_queue':txs_queue,
              'unsigned_txs_queue':unsigned_txs_queue} 
    metrics = {'last_block_index':0, #?
               'miners_fees':{'tau':0, 'avg':0,'min':np.inf,'max':-np.inf, 'sum':0},
               'block_pow':{'tau':0, 'avg':0,'min':np.inf,'max':-np.inf, 'sum':0},
               'block_txs_count':{'tau':0, 'avg':0,'min':np.inf,'max':-np.inf, 'sum':0},
               'crval':{'tau':0, 'avg':0,'min':np.inf,'max':-np.inf, 'sum':0},
               'unsigned_txs':{'tau':0, 'avg':0,'min':np.inf,'max':-np.inf, 'sum':0}
              }
    
    return (queues, metrics)